In [ ]:
import utils
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
import matplotlib.pyplot as plt

# Data loading

In [ ]:
classes = os.listdir('out')
csv_list = os.listdir('out/handshake')

In [ ]:
class_videos = {}
total = []

factorize_classes = {_class: key for (_class, key) in zip(classes, range(len(classes)))}

idx = 0

for _class in classes:
    if os.path.isdir('out/'+_class):
        class_videos[_class] = os.listdir('out/'+_class)

for _class, files in class_videos.items():
    for file in files:
        if os.path.isfile("out/handshake/"+file):
            if file.split('.')[-1] == 'csv':
                csv = pd.read_csv("out/handshake/"+file)
                
                file_id = int(file.split('.')[0].lstrip('0'))
                video_number = pd.DataFrame({'_id': idx, 'video':[file_id]*csv.shape[0]})

                csv['result'] = factorize_classes[_class]
                total.append(pd.concat([video_number, csv], axis=1))
                idx += 1


In [ ]:
result = pd.concat(total, ignore_index=True)

# Data preprocessing

In [ ]:
score_columns = list(filter(lambda x: x.endswith('score'), list(result.columns)))

In [ ]:
result = result.fillna(0)

In [ ]:
result.iloc[:, 4:-1] = result.iloc[:, 4:-1].replace(0, -1)

In [ ]:
result

In [ ]:
(result.iloc[:, 4:-1] == 0).sum().sum() / result.iloc[:, 4:-1].size

In [ ]:
print('Proportion of NAs cells in dataset: ' + str((result.iloc[:, 4:-1] == -1).sum().sum() / result.iloc[:, 4:-1].size))
print('Proportion of NAs rows in dataset: ' + str((result.iloc[:, 4:-1] == -1).sum(1).count() / result.iloc[:, 4:-1].shape[0]))

In [ ]:
(result[score_columns] == -1).sum().sort_values(ascending=False)

In [ ]:
# columns with parts of lower body which might poorly contribute to prediction of interactions
to_drop = list(filter(lambda x: x.find('Ankle') != -1 or x.find('Hip') != -1 or x.find('Knee') != -1, list(result.columns)))

In [ ]:
result.drop(score_columns, axis=1, inplace=True)
#result.drop(set(to_drop)-set(score_columns), axis=1, inplace=True)

In [ ]:
result.columns

In [ ]:
result_array = np.array(result)